In [2]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/hubble/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/hubble/')
import os
os.environ['PATH'] = '/Users/ravi.tej/anaconda3/envs/bertopicenv/bin:/Users/ravi.tej/anaconda3/condabin:/usr/bin:/bin:/usr/sbin:/sbin'

In [3]:
import anthropic

In [4]:
client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.environ.get('ANTHROPIC_KEY'),
)

In [1]:
prompt = '''As a financial analyst, your role is to iteratively summarize the annual reports of Indian companies. You will receive the report in 20-page increments and need to distill this into a comprehensive summary, aiming for a final length of 20-30 pages. At each step, you will know how many pages you have already summarized and have access to the existing summary.

**Objective**: Extract essential information from each 20-page section to develop a concise yet thorough summary of the company's financial and strategic status.

**Process and Instructions**:

1. **Review Incrementally**: With each 20-page increment, assess the content for critical information relevant to the following categories. Update the summary progressively, ensuring you maintain a cohesive narrative throughout the document.

2. **Executive Summary (1-2 pages in final summary)**: Extract key points from the CEO/Chairman’s message, focusing on the company’s strategic direction, major achievements, and future priorities.

3. **Financial Highlights (2-3 pages)**: Identify crucial financial metrics, trends, and year-over-year changes. Maintain a running tally of these metrics to easily integrate into the summary.

4. **Business Overview (3-4 pages)**: Summarize core business activities, market presence, and significant corporate actions, such as mergers or acquisitions.

5. **Market Analysis (3-4 pages)**: Capture insights on market trends, competitive landscape, and external factors impacting the business.

6. **Financial Performance Analysis (4-5 pages)**: Provide a detailed review of the financial statements, noting significant items and variances from projections or benchmarks.

7. **Risk Management and Corporate Governance (2-3 pages)**: Outline the company’s risk landscape and governance practices, emphasizing major changes or developments.

8. **Sustainability and CSR (2 pages)**: Document the company's ESG efforts, including any quantitative metrics or achievements.

9. **Future Outlook (2 pages)**: Note forward-looking statements regarding the company’s strategic plans, investment initiatives, and potential market or operational challenges.

10. **Page Count Management**: After summarizing each segment, check the accumulated page count. This awareness will guide you to appropriately pace the remaining sections and ensure that critical areas are covered without breaching the 30-page limit.

11. **Iterative Refinement**: After completing each section summary, revisit the overall narrative to ensure consistency and coherence. Adjust as necessary to maintain a focused and integrated summary.

12. **Prioritization and Conciseness**: Given the iterative nature of the task, prioritize information that adds significant insight or value to the summary. Aim for brevity and clarity, avoiding redundancy or less critical details.

**Effective Strategies**:

- Maintain flexibility in the summary process, allowing for adjustments based on the content's significance and the evolving narrative.
- Use the ongoing access to the summarized content to ensure that the information flows logically and maintains thematic consistency.
- Regularly cross-check the developing summary against the total page count to manage space effectively and ensure that all critical areas are comprehensively covered.

By following these guidelines, you can produce a detailed and coherent summary that captures the essential aspects of the company's annual performance and strategic direction, staying within the desired page limit.'''

In [8]:
summary_so_far = ''

In [12]:
file = 'Zomato Annual Report 2022-23.pdf'

In [6]:
import fitz # imports the pymupdf library
doc = fitz.open("Zomato Annual Report 2022-23.pdf") # open a document

In [7]:
txt = []
for page in doc: # iterate the document pages
    txt.append(page.get_text()) # get plain text encoded as UTF-8

In [10]:
cur_text = ''.join(txt[:30])

In [27]:
import numpy as np

13.0

In [14]:
len(txt)

369

In [15]:
input_prompt = f'<instructions>: {prompt}. <annual_report_title>: {file}. <total_pages>: {len(txt)}. <current_provided_pages>: {0} to {30}. <summary_so_far>: {summary_so_far}. <current_pages_report>: {cur_text}'

In [18]:
message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=4096,
    messages=[
        {"role": "user", "content": input_prompt}
    ]
)

In [47]:
num_pages = len(txt)
page_batch = 30
num_loops = int(np.ceil(num_pages/page_batch))

In [36]:
responses = []
summary_so_far = ''

In [40]:
from tqdm.notebook import tqdm

In [43]:
from time import sleep

In [44]:
loop_num

4

In [46]:
responses

[Message(id='msg_01HRszQUSZPzXiu1ySNVwFzd', content=[ContentBlock(text="Here are the key points summarizing Zomato's progress on ESG initiatives in FY23:\n\nClimate Conscious Deliveries:\n- Committed to achieving net zero emissions across food delivery value chain by 2033\n- Set targets to reduce last-mile delivery emissions per km by 70% by FY30 and maintain net zero Scope 1 & 2 emissions\n- Took actions like purchasing carbon offsets, exploring renewable energy solutions, and partnering to onboard 100,000 EV delivery partners by 2025\n\nWaste Free World: \n- Announced 100% plastic neutral deliveries, recycled over 20,000 MT plastic waste in FY23 (2x the plastic used)\n- Launched re-usable packaging pilot, committed to 100+ million eco-friendly deliveries by 2025\n- Blinkit moved completely to paper-based packaging\n\nZero Hunger:\n- Hosted Zomato Feeding India Concert to raise awareness on hunger/malnutrition\n- Feeding India initiative provided meals to underprivileged groups  \n\nS

In [48]:
for loop_num in tqdm(range(4, num_loops)):
    page_start = loop_num * page_batch
    page_end = min(num_pages, (loop_num + 1) * page_batch)
    cur_text = ''.join(txt[page_start:page_end])
    cur_input_prompt = f'''<instructions>: {prompt}. 
                            <annual_report_title>: {file}. 
                            <total_pages>: {len(txt)}. 
                            <current_provided_pages>: {page_start} to {page_end}. 
                            <summary_so_far>: {summary_so_far}. 
                            <current_pages_report>: {cur_text}'''
    message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=4096,
    messages=[
        {"role": "user", "content": input_prompt}
    ]
    )
    responses.append(message)
    summary_so_far += message.content[0].text
    sleep(60)

  0%|          | 0/9 [00:00<?, ?it/s]

In [49]:
summary_so_far

"Here are the key points summarizing Zomato's progress on ESG initiatives in FY23:\n\nClimate Conscious Deliveries:\n- Committed to achieving net zero emissions across food delivery value chain by 2033\n- Set targets to reduce last-mile delivery emissions per km by 70% by FY30 and maintain net zero Scope 1 & 2 emissions\n- Took actions like purchasing carbon offsets, exploring renewable energy solutions, and partnering to onboard 100,000 EV delivery partners by 2025\n\nWaste Free World: \n- Announced 100% plastic neutral deliveries, recycled over 20,000 MT plastic waste in FY23 (2x the plastic used)\n- Launched re-usable packaging pilot, committed to 100+ million eco-friendly deliveries by 2025\n- Blinkit moved completely to paper-based packaging\n\nZero Hunger:\n- Hosted Zomato Feeding India Concert to raise awareness on hunger/malnutrition\n- Feeding India initiative provided meals to underprivileged groups  \n\nSustainable Livelihoods:\n- Provided earning opportunities to 326k avera

In [21]:
message.content[0].text

"Here are the key points summarizing the progress on Zomato's sustainability initiatives in FY23:\n\nClimate Conscious Deliveries:\n- Committed to achieving net zero emissions across food delivery value chain by 2033\n- Purchased carbon offsets for Scope 1 & 2 emissions in FY23\n- Committed to 100% EV-based food deliveries by 2030 as part of EV100 initiative\n- Partnered with over 50 EV companies, onboarded ~13,500 EV delivery partners in FY23\n\nWaste Free World:  \n- Recycled over 20,000 MT of plastic waste in FY23, more than 2x the plastic used in deliveries\n- Launched reusable packaging pilot in Bengaluru \n- Goal of 100+ million eco-friendly (plastic-free) deliveries by 2025\n\nZero Hunger:\n- Feeding India initiative provided meals to underprivileged, hosted Zomato Feeding India Concert\n- Launched campaign with Unacademy to support children who lost parents in pandemic\n\nSustainable Livelihoods:\n- Provided earning opportunities to 326k average monthly active delivery partners